In [1]:
import argparse
import pandas as pd
import numpy as np
import math
import h5py
from sklearn.model_selection import train_test_split
import joblib
import pickle
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import roc_curve, auc
import tensorflow as tf
import sys
import gc
import logging
import keras_tuner as kt
import os

# import setGPU
import tensorflow.keras as keras
import tensorflow_model_optimization as tfmot
tsk = tfmot.sparsity.keras
from tensorflow.keras import layers
from tensorflow.keras import backend as K
#tf.keras.mixed_precision.set_global_policy('mixed_float16')
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, TerminateOnNaN
from tensorflow.keras.layers import (
    Lambda,
    Input,
    Dense,
    Conv2D,
    AveragePooling2D,
    MaxPooling2D,
    UpSampling2D,
    ZeroPadding2D,
    Conv2DTranspose,
    BatchNormalization,
    Flatten,
    Reshape,
    Activation,
    ReLU,
    LeakyReLU,
    Dropout,
    Concatenate,
    Cropping1D,
    Layer,
    )

from datetime import datetime
from tensorboard import program
import os
import pathlib
import matplotlib as mpl
import matplotlib.pyplot as plt
try:
    import mplhep as hep
    hep.style.use(hep.style.ROOT)
    print("Using MPL HEP for ROOT style formating")
except:
    print("Instal MPL HEP for style formating")
mpl.rcParams['axes.prop_cycle'] = mpl.cycler(color=["#DB4437", "#4285F4", "#F4B400", "#0F9D58", "purple", "goldenrod", "peru", "coral","turquoise",'gray','navy','m','darkgreen','fuchsia','steelblue']) 

#from autoencoder_classes import AE,VAE
#from neptunecontrib.monitoring.keras import NeptuneMonitor
#from losses import mse_split_loss, radius, kl_loss
#from functions import make_mse_loss_numpy
#from data_preprocessing import prepare_data
#from model import build_AE, build_VAE, Sampling

def return_total_loss(loss, bsm_t, bsm_pred):
    total_loss = loss(bsm_t, bsm_pred.astype(np.float32))
    return total_loss

#from qkeras.quantizers import quantized_bits
from keras.utils import tf_utils
quantize=False

import time
ktuner_results = f"{int(time.time())}"

Using MPL HEP for ROOT style formating


In [2]:
def mse_loss_numpy(inputs, outputs):
    
    # remove last dimension
    inputs = np.reshape(inputs, (inputs.shape[0],19,3))
    outputs = np.reshape(outputs, (outputs.shape[0],19,3)) 
    
    mask0 = inputs[:,:,0]!=0
    mask1 = inputs[:,:,1]!=0
    mask2 = inputs[:,:,2]!=0
    mask = (mask0 + mask1 + mask2)*1
    mask = np.reshape(mask, (mask.shape[0],19,1))
    inputs = inputs*mask
    outputs = outputs*mask

    # remove zero entries
    loss = np.mean(np.square(inputs.reshape(inputs.shape[0],57)-outputs.reshape(outputs.shape[0],57)),axis=1)
    return loss

def mse_loss(inputs, outputs):
    # remove last dimension
    inputs = tf.reshape(inputs, (tf.shape(inputs)[0],19,3))
    outputs = tf.reshape(outputs, (tf.shape(outputs)[0],19,3))
    
    mask0 = tf.math.not_equal(inputs[:,:,0],0)
    mask1 = tf.math.not_equal(inputs[:,:,1],0)
    mask2 = tf.math.not_equal(inputs[:,:,2],0)
    mask = tf.math.logical_and(mask0, mask1)
    mask = tf.math.logical_and(mask, mask2)
    # tf.print(mask)
    mask = tf.cast(mask, tf.float32)
    mask = tf.reshape(mask, (tf.shape(mask)[0],19,1))

    # remove zero entries
    loss = reco_scale*tf.reduce_mean(tf.square(inputs[:,:,:]-outputs[:,:,:])*mask)
    return loss

def kl_loss(mu, logvar, beta=None):
    kl_loss = 1 + logvar - np.power(mu, 2) - np.exp(logvar)
    kl_loss = np.mean(kl_loss, axis=-1) # mean over latent dimensions
    kl_loss *= -0.5
    if beta!=None: return beta*kl_loss
    else: return kl_loss

def make_kl(z_mean, z_log_var):
    @tf.function
    def kl_loss(inputs, outputs):
        kl =  - 0.5 * (1. + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
        kl = tf.reduce_mean(kl, axis=-1) # multiplying mse by N -> using sum (instead of mean) in kl loss (todo: try with averages)
        return kl
    return kl_loss

class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

def make_mse_kl(z_mean, z_log_var, beta):
    kl_loss = make_kl(z_mean, z_log_var)
    # multiplying back by N because input is so sparse -> average error very small
    @tf.function
    def mse_kl_loss(inputs, outputs):
        return mse_loss(inputs, outputs) + kl_loss(inputs, outputs) if beta==0 \
            else (1 - beta) * mse_loss(inputs, outputs) + kl_loss(inputs, outputs)
    return mse_kl_loss

#mse = tf.keras.losses.MeanSquaredError()

def total_objective(encoder, vae, qcd_data, bsm_data):
    def total_objective_val(y_true, y_pred):
        # Evaluate total loss for the two datasets
        # First compute KL losses 
        z_mean_qcd = encoder.predict(qcd_data)[0]
        z_logvar_qcd = encoder.predict(qcd_data)[1]
        kl_loss_qcd = kl_loss(z_mean_qcd, z_logvar_qcd)

        z_mean_bsm = encoder.predict(bsm_data)[0]
        z_logvar_bsm = encoder.predict(bsm_data)[1]
        kl_loss_bsm = kl_loss(z_mean_bsm, z_logvar_bsm)

        # Then compute reconstruction loss (MSE) 
        qcd_predict = vae.predict(qcd_data)
        reco_loss_qcd = mse_loss_numpy(tf_utils.sync_to_numpy_or_python_type(qcd_data), tf_utils.sync_to_numpy_or_python_type(qcd_predict))

        bsm_predict = vae.predict(bsm_data)
        reco_loss_bsm = mse_loss_numpy(tf_utils.sync_to_numpy_or_python_type(bsm_data), tf_utils.sync_to_numpy_or_python_type(bsm_predict))

        # Total losses are
        total_loss_qcd = kl_loss_qcd + reco_loss_qcd
        total_loss_bsm = kl_loss_bsm + reco_loss_bsm

        total_true_val = np.concatenate((np.ones(total_loss_bsm.shape[0]), np.zeros(total_loss_qcd.shape[0])))
        total_pred_val = np.nan_to_num(np.concatenate((total_loss_bsm, total_loss_qcd)))

        total_fpr_loss, total_tpr_loss, total_threshold_loss = roc_curve(total_true_val, total_pred_val)
        total_objective = np.interp(10**(-5), total_fpr_loss, total_tpr_loss)
        
        return total_objective
    return total_objective_val

In [3]:
class VAE(kt.HyperModel):
    
    def __init__(self, beta, qcd_target, bsm_target):
        self.beta = beta
        self.qcd_target = qcd_target
        self.bsm_target = bsm_target
        
    def build(self, hp):

        input_shape=57
        inputs = keras.Input(shape=(input_shape,))
        before_bottleneck = 0        
        
        W = hp.Int("first_layer_width", min_value=16, max_value=256, step=16) # width of first layer
        num_layers = hp.Int("number of layers", 2, 5) # number of layers
        z_width = hp.Int("latent_dim_width", min_value=2, max_value=12, step=1)

        fixed_step = W/num_layers # decrease/increase subsequent encoder/decoder layers by this amount 

        x = layers.Dense(W,kernel_initializer='lecun_uniform', activation='relu')(inputs) # create first layer
        
        for i in range(num_layers - 1): # subsequent encoder layer widths decrease in size by i*fixed_step
            before_bottleneck = W - (i+1)*fixed_step
            x = layers.Dense(int(before_bottleneck), kernel_initializer='lecun_uniform', activation='relu')(x)

        z_mean = layers.Dense(z_width,kernel_initializer=tf.keras.initializers.HeUniform(seed=42))(x)
        z_logvar = layers.Dense(z_width,kernel_initializer=tf.keras.initializers.HeUniform(seed=42))(x)
        z = Sampling()([z_mean, z_logvar])
        encoder = keras.Model(inputs, [z_mean, z_logvar, z], name="encoder")
        encoder.summary()

        latent_inputs = keras.Input(z_width,)
        y = layers.Dense(int(before_bottleneck), kernel_initializer='lecun_uniform', activation='relu')(latent_inputs)

        for i in range(1, num_layers - 1):
            after_bottleneck = before_bottleneck + i*fixed_step
            y = layers.Dense(int(after_bottleneck),kernel_initializer='lecun_uniform', activation='relu')(y)

        y = layers.Dense(W,kernel_initializer='lecun_uniform', activation='relu')(y)
        decoded = layers.Dense(input_shape)(y)
        decoder = keras.Model(latent_inputs, decoded, name="decoder")
        decoder.summary()

        vae_outputs = decoder(encoder(inputs)[2])
        vae = keras.Model(inputs, vae_outputs, name = 'vae')
        vae.summary()

        obj = total_objective(encoder, vae, self.qcd_target, self.bsm_target)

        vae.compile(optimizer=keras.optimizers.Adam(),
                    loss=make_mse_kl(z_mean, z_logvar, self.beta),
                    metrics=[mse_loss, make_kl(z_mean, z_logvar), obj])

        return vae

In [4]:
# Sample layer generations
W = 16
num_layers = 3
fixed_step = W/num_layers

for i in range(num_layers-1):
    before = W - (i+1)*fixed_step
    print(before)

print('<put latent dimension layers here>')
    
for i in range(num_layers-1):
    after = before + i*fixed_step
    print(after)

10.666666666666668
5.333333333333334
<put latent dimension layers here>
5.333333333333334
10.666666666666668


In [5]:
def optimization(input_qcd, input_bsm, beta):
    
    # magic trick to make sure that Lambda function works
    tf.compat.v1.disable_eager_execution()
    
    output={}
    
    with h5py.File(input_qcd, 'r') as h5f:
        output['ZeroBias'] = {}
    
        data = np.array(h5f['full_data_cyl'][:events], dtype=np.float32)
        ET = np.array(h5f['ET'][:events], dtype=np.float32)
        L1bit = np.array(h5f['L1bit'][:events], dtype=np.int8)

        #mask saturated ET
        mask_ET = ET<2047.5
        ET = ET[mask_ET]
        data = data[mask_ET]
        L1bit = L1bit[mask_ET]
    
        #mask saturated PT
        mask_0  = data[:,0,0]<2047.5
        mask_1_9  = data[:,1:9,0]<255.5
        mask_9_20  = data[:,9:20,0]<1023.5
        mask = np.concatenate((mask_0[:,np.newaxis],mask_1_9,mask_9_20),axis=1)*1
        data = data*mask[:,:,np.newaxis]

        pt = np.copy(data[:,:,0])
        eta = np.copy(data[:,:,1])
        phi = np.copy(data[:,:,2])
    
        data[:,:,0] = pt*np.cos(phi)
        data[:,:,1] = pt*np.sin(phi)
        data[:,:,2] = pt*np.sinh(eta)
        data_target = np.copy(data)

        del pt, eta, phi, mask_ET, mask_0, mask_1_9, mask_9_20, mask
    
        if(norm=='ET'):
            data_target[:,:,:] = data[:,:,:]/ET[:,None,None]
            std_xy = (np.std(data_target[:,:,0])+np.std(data_target[:,:,1]))/2
            std_z = np.std(data_target[:,:,2])
            data_target[:,:,2] = data_target[:,:,2]*(std_xy/std_z)
        elif(norm=='std'):
            mean_qcd = np.mean(data_target, axis=0)
            std_qcd = np.std(data_target, axis=0)
            data_target = (data_target[:,:,:] - mean_qcd[None,:,:])/std_qcd[None,:,:]

            # mean_qcd = np.array([np.mean(data_target[:,:,0]),np.mean(data_target[:,:,1]),np.mean(data_target[:,1:20,2])])
            # std_qcd = np.array([np.std(data_target[:,:,0]),np.std(data_target[:,:,1]),np.std(data_target[:,1:20,2])])
            # data_target[:,:,0] = (data_target[:,:,0]-mean_qcd[0])/std_qcd[0]
            # data_target[:,:,1] = (data_target[:,:,1]-mean_qcd[1])/std_qcd[1]
            # data_target[:,:,2] = (data_target[:,:,2]-mean_qcd[2])/std_qcd[2] 
            data_target[:,0,2] = 0
        else:
            data_target[:,0,:] = data[:,0,:]/2048
            data_target[:,1:9,:] = data[:,1:9,:]/256
            data_target[:,9:20,:] = data[:,9:20,:]/1024
        

        X_train, output['ZeroBias']['data'], Y_train, output['ZeroBias']['target'], _ , output['ZeroBias']['ET'], _ ,output['ZeroBias']['L1bit'] =  train_test_split( data, data_target, ET,L1bit, test_size=0.5)

        X_train = X_train.reshape(X_train.shape[0], X_train.shape[1]*X_train.shape[2])
        Y_train = Y_train.reshape(Y_train.shape[0], Y_train.shape[1]*Y_train.shape[2])

        del data, data_target, ET, L1bit
        
    with h5py.File(input_bsm,'r') as h5f2:
        for key in h5f2.keys():
            if('TT' not in key[:2]) and ('haa4b_ma15_powheg' not in key) and ('GluGluToHHTo4B_cHHH1' not in key): continue
            if len(h5f2[key].shape) < 3: continue
            
            output[str(key)] = {}
            output[str(key)]['data'] = np.array(h5f2[str(key)][:events,:,:],dtype=np.float32)
            output[str(key)]['ET'] = np.array(h5f2[str(key)+'_ET'][:events],dtype=np.float32)
            output[str(key)]['L1bit'] = np.array(h5f2[str(key)+'_l1bit'][:events],dtype=np.int8)

            #mask saturated ET
            mask_ET = output[str(key)]['ET']<2047.5
            output[str(key)]['ET'] = output[str(key)]['ET'][mask_ET]
            output[str(key)]['data'] = output[str(key)]['data'][mask_ET]
            output[str(key)]['L1bit'] = output[str(key)]['L1bit'][mask_ET]
        
            #mask saturated PT
            mask_0  = output[str(key)]['data'][:,0,0]<2047.5
            mask_1_9  = output[str(key)]['data'][:,1:9,0]<255.5
            mask_9_20  = output[str(key)]['data'][:,9:20,0]<1023.5
            mask = np.concatenate((mask_0[:,np.newaxis],mask_1_9,mask_9_20),axis=1)*1
            output[str(key)]['data'] = output[str(key)]['data']*mask[:,:,np.newaxis]

            pt = np.copy(output[str(key)]['data'][:,:,0])
            eta = np.copy(output[str(key)]['data'][:,:,1])
            phi = np.copy(output[str(key)]['data'][:,:,2])
        
            output[str(key)]['data'][:,:,0] = pt*np.cos(phi)
            output[str(key)]['data'][:,:,1] = pt*np.sin(phi)
            output[str(key)]['data'][:,:,2] = pt*np.sinh(eta)

            del pt, eta, phi, mask_ET, mask_0, mask_1_9, mask_9_20, mask


            output[str(key)]['target'] = np.copy(output[str(key)]['data'])
            if(norm=='ET'):
                output[str(key)]['target'] = output[str(key)]['data']/output[str(key)]['ET'][:,None,None]
                output[str(key)]['target'][:,:,2] = output[str(key)]['target'][:,:,2]*(std_xy/std_z)
            elif(norm=='std'):
                output[str(key)]['target'] = (output[str(key)]['target'] - mean_qcd[None,:,:])/std_qcd[None,:,:]
                # output[str(key)]['target'][:,:,0]= (output[str(key)]['data'][:,:,0]-mean_qcd[0])/std_qcd[0]
                # output[str(key)]['target'][:,:,1]= (output[str(key)]['data'][:,:,1]-mean_qcd[1])/std_qcd[1]
                # output[str(key)]['target'][:,:,2]= (output[str(key)]['data'][:,:,2]-mean_qcd[2])/std_qcd[2]
                output[str(key)]['target'][:,0,2] = 0
            elif(norm=='max_PT'):
                output[str(key)]['target'][:,0,:] = output[str(key)]['data'][:,0,:]/2048
                output[str(key)]['target'][:,1:9,:] = output[str(key)]['data'][:,1:9,:]/256
                output[str(key)]['target'][:,9:20,:] = output[str(key)]['data'][:,9:20,:]/1024
                
    qcd_target = np.copy(output['ZeroBias']['target'])
    qcd_target = qcd_target.reshape(qcd_target.shape[0],57)
    bsm_target = np.copy(output['haa4b_ma15_powheg']['target'])
    bsm_target = bsm_target.reshape(bsm_target.shape[0],57)
    print('qcd_target is a ', type(qcd_target))
    
    hypermodel = VAE(beta, qcd_target, bsm_target)
    
    ktuner = kt.BayesianOptimization(
            hypermodel,
            objective = kt.Objective('val_total_objective_val', direction='min'),
            max_trials = 1,
            executions_per_trial = 3,
            directory = ktuner_results)
    
    ktuner.search_space_summary()
    
    ktuner.search(x=X_train,
                 y=Y_train,
                 epochs=150,
                 batch_size=1024,
                 validation_split=0.2,
                 callbacks=[tf.keras.callbacks.EarlyStopping('val_loss',patience=5)])
    
    with open(f"ktuner_{int(time.time())}.pkl", "wb") as f:
        pickle.dump(ktuner, f)
    
    ktuner.results_summary()
    
    logging.info('Get the optimal hyperparameters')
    best_hps = ktuner.get_best_hyperparameters(num_trials=5)[0]
    logging.info('Getting and printing best hyperparameters!')
    print(best_hps)
        

In [6]:
input_hardqcd="/eos/uscms/store/group/lpctrig/jngadiub/L1TNtupleRun3-h5-extended-v2/QCD_preprocessed.h5"
input_qcd="/eos/uscms/store/group/lpctrig/jngadiub/L1TNtupleRun3-ZB-h5-extended-v2/ZB_preprocessed.h5"
input_bsm = "/eos/uscms/store/group/lpctrig/jngadiub/L1TNtupleRun3-h5-extended-v2-120X/BSM_preprocessed.h5"
events=5000000
norm = 'std'
beta = 0.8
reco_scale = 1000

In [ ]:
optimization(input_qcd, input_bsm, beta)